# **Projekt: Fehlererkennung in Getrieben**
## **Netztyp 2: FFT-Net**


Mey et al. (2020) untersuchten die automatisierte Erkennung von Unwuchten in rotierenden Wellen mithilfe von Maschinellem Lernen und veröffentlichten hierfür einen spezifischen Datensatz. In diesem Kontext entwickeln und analysieren sie vier Klassifizierungsalgorithmen: Convolutional Neural Network (CNN), Fully-Connected Neural Network (FCNN), Hidden Markov Model und Random Forest. Diese Algorithmen unterscheiden zwischen den Zuständen "eine Unwucht liegt vor" und "keine Unwucht liegt vor". Das FCNN erreicht hier, als die erfolgreichste Methode, eine Vorhersagegenauigkeit von 98,5 %. <br>
Das vorliegende Projekt reproduziert zunächst das von Mey et al. beschriebene CNN. Auf Basis des FCNNs wird zudem ein Neuronales Netzwerk (NN) entwickelt, das eine Unterscheidung der im Datensatz angegebenen fünf Unwucht-Stufen leisten kann. Das CNN verarbeitet hierbei direkt Sensordaten. Für das FCNN überführt die Fast
Fourier Transform (FFT) die Daten vorab in den Frequenzbereich. <br>
Im Rahmen des Projektes sind so mehrere Python-Module entstanden. Der vollständige Code ist in einem GitHub-Repository (Kuhlen et al., 2024) verfügbar.

<details>
    <summary open><b>Abkürzungsverzeichnis</b></summary>

> <br>
>
> | Abkürzung | Bedeutung |
> |:----------|:----------|
> | CNN | Convolutional Neural Network |
> | CSV | Comma-Separated Values |
> | DFT | Discrete Fourier-Transform, Diskrete Fourier-Transformation |
> | FCNN | Fully Connected Neural Network |
> | FFT | Fast Fourier Transform |
> | IQR | Inter-Quantile Range, Interquantilabstand |
> | NN | Neural Network, Neuronales Netzwerk |
> | UUID | Universally Unique Identifier |
> | YAML | Yet Another Markdown Language |
>
> <br>
</details>


### **Teil A: Preprocessing**

Mey et al. (2020) beschreiben ausschließlich die wesentlichen Schritte der Datenaufbereitung, dem Preprocesing. Ihre Ausführungen werden um Funktionalitäten ergänzt. Deren Notwendigkeit ist an entsprechender Stelle begründet.

#### **Die Projektstruktur**
Das FFT-Net gliedert sich in die zwei Jupyter Notebooks *fft_net_preprocessing.ipynb* und *fft_net.ipynb*. Die *fft_net_preprocessing.ipynb* umfasst den Download des Datensatzes und das eigentliche Preprocessing. Die aufbereiteten Daten greift die *fft_net.ipynb* auf. Sie erstellt, trainiert und bewertet das NN. Im Projekt finden sich diese Skripte im Ordner *working_dir/notebooks/*. Auf die Nennung des Working Directories *working_dir* wird in nachfolgenden Dateipfaden verzichtet. <br>
Auf dem Pfad *src/* befinden sich die vier Python-Skripte *data_loading.py*, *input_preparation.py*, *model_construction.py*, und *visualization.py* und der Ordner *preprocessing/*. Jedes dieser Module umfasst Funktionen für einen spezifischen Prozessschritt hin zu einem funktionsfähigen NN. Sie sind im Folgenden in chronologischer Reihenfolge beschrieben. <br>
Das Modul `data_loading` umfasst Funktionen zum Herunterladen des Kaggle-Datensatzes, sowie zum Laden und Speichern von Datensätzen aus der und in die Projekt-Ordnerstruktur. Im Modul `preprocessing` sind alle Funktionen für die Vorverarbeitung der Daten enthalten. Die Vorbereitung der vom Preprocessing aufbereiteten Daten auf die Eingabe in das Neuronale Netz übernehmen Funktionen aus der `input_preparation`. Das eigentliche NN konstruiert der Constructor aus der `model_construction`. Hier sind zudem weitere Funktionen mit Bezug auf die Architektur und Performance des NNs abgelegt. Das Modul `visualization` bietet Funktionen zur Visualisierung von Daten über den gesamten Prozess.

In [ ]:
# import libraries
from uuid import uuid4
import os
from pathlib import Path

# module imports from /src
if Path.cwd().stem == "notebooks":
    os.chdir( Path.cwd().parent)

# custom modules
import src.preprocessing as pre
import src.data_loader as dl
import src.visualization as vis

Zu Beginn des Codes erlauben einige Booleans die Kontrolle des Prozessablaufs. Das Notebook erlaubt die Kontrolle der korrekten Funktionsweise über einen Testbetrieb, der mit der Boolean `TEST` aktiviert wird. Die Boolean `DOWNLOAD` aktiviert den (erneuten) Download des Kaggle Datensatzes. Ist die Visualisierung der Daten im Prozess nicht gewünscht, deaktiviert die Boolean `SHOW` die entsprechenden Funktionen. Über einen hier gesetzten `CONTROL_INDEX` erfolgt im Prozess die stichprobenartige Visualisierung der Daten. An dieser Stelle wird zudem ein Universally Unique Identifier (`UUID`) generiert, der bei der Speicherung der verarbeiteten Daten als Ordnername verwendet wird. <br>

In [ ]:
# boolean values to control the program flow
TESTING: bool = True
DOWNLOAD: bool = False
SHOW: bool = True

# setup a single index for control of preprocessing steps
CONTROL_INDEX: int = 4

# generate a unique identifier for this run
UUID: str = str(uuid4())
print(f"Current UUID:\t{UUID}")

#### **Der Datensatz**

Die Datenerfassung beschreiben Mey et al. (2020) wie folgt. Eine Welle wird von einem Gleichstrommotor angetrieben (vergl. Abb. 1). Sie ist in Richtung des freien Endes in einem Kugellager gelagert. In der Ebene des Kugellagers sitzen zwei Sensoren, die die Vibration der Welle in der Horizontalen und Vertikalen erfassen. Ein dritter Sensor sitzt oberhalb der Einspannung der Welle am Motor. Die Unwucht wird über eine Schwungscheibe am freien Wellenende in das System eingebracht. Auf dieser Scheibe können Schrauben auf verschiedenen Radien montiert werden. <br>
Mey et al. erhöhten pro Messreihe ist die Eingangsspannung beginnend von einem Startwert in festen Intervallen bis ein Endwert erreicht wurde. Die jeweiligen Start- und Endwerte sowie die Intervallgrößen sind bei Mey et al. bzw. in der *data/raw/meta.yaml* nachzulesen. Jeder Spannungswert ist für 10 s gehalten worden. Die erzeugten Rotationsgeschwindigkeiten reichen von rund 630 RPM bis ca. 2330 RPM. <br>
Jede Messreihe ist mit einer Sample Rate von 4096 Hz aufgezeichnet worden. Es wurden insgesamt vier Unwuchtstärken erzeugt. Als Referenz ist eine Messung ohne eingebrachte Unwucht aufgezeichnet worden. Neben den Messungwerten der drei Sensoren ist in jedem Datensatz der Verlauf der Spannung und der Drehzahl enthalten.

In [ ]:
if DOWNLOAD:
    
    # download full dataset from kaggle
    dl.fetch_kaggle_dataset(dl.DATASET_ADDRESS)

Sobald der Download des Datensatzes abgeschlossen ist, liegen die CSV Dateien auf dem Pfad *working_dir/data/raw/*. Auf diesem Pfad liegt derweil bereits eine YAML Datei, die Informationen zu den Aufnahmebedingungen jeder Messreihe enthält. Die Funktion `load_all_datasets()` greift auf den Dateipfad zu und erstellt aus allen CSV Dateien Pandas DataFrames. In deren `DataFrame.attrs` hinterlegt sie die korrespondierenden Informationen der YAML Datei.

In [ ]:
if not TESTING:
    
    # load data from folder and split in training and evaluation data
    data_path = Path().cwd() / "data" / "raw"
    development_data, evaluation_data = dl.load_all_datasets(data_path)
    all_datasets = development_data + evaluation_data

else:

    # program testing set
    test = dl.load_dataset(Path("./data/raw/3D.csv"))

In [ ]:
if SHOW:
    if not TESTING:

        for index, dataset in enumerate(all_datasets):
            # visualize one random dataset for data validation
            print(dataset.attrs["path"])
            vis.plot_columns_as_subplots(dataset, dataset.columns)

    else:

        # visualize the program testing set
        vis.plot_columns_as_subplots(test, test.columns)

#### **Die Datenaufbereitung**
##### **Bereinigung der Daten**
Jeder Datensatz zeigt zu Beginn der Sensordaten einen starken positiven Ausschlag. Die ersten rund 12.000 aufgezeichneten Spannungen liegen nicht auf dem Startwert, sondern betragen null. Thresholding entfernt zunächst alle Datenpunkte, die bei 0 V aufgezeichnet wurden. Die erneute Visualisierung der Daten zeigt, dass der positive Ausschlag der Sensordaten so nicht vollständig beseitigt ist. Daher verwirft das Programm weitere ca. 38.000 Zeilen. Damit folgt das Preprocessing den Ausführungen von Mey et al. (2020) und verwirft die ersten 50.000 Messpunkte.

In [ ]:
if not TESTING:

    for index, dataset in enumerate(all_datasets):
        pre.apply_threshold(
            dataframe=dataset,
            threshold=0,
            column="V_in",
            mode="eq",
            copy=False,
            reset_index=False,
        )
        dataset = pre.discard_data(
            dataframe=dataset, start=None, end=50_000, reset_index=True
        )

else:

    pre.apply_threshold(
        dataframe=test,
        threshold=0,
        column="V_in",
        mode="eq",
        copy=False,
        reset_index=False,
    )
    test = pre.discard_data(dataframe=test, start=None, end=50_000, reset_index=True)

Anhand des Verlaufs der Eingangsspannung erfolgt die Zerlegung eines jeden DataFrames in die jeweiligen beiden Messzyklen. Als Schnittpunkte dienen hierbei die negativen Gradienten in den Spannungen. Bei diesem Prozess entstehen z. T. kurze Residualelemente, die für die weitere Bearbeitung entfernt werden. In den `DataFrame.attrs` wird dem Dateinamen eine Laufnummer mit vorangestelltem Unterstrich angehangen.

In [ ]:
if not TESTING:

    measurement_cycles = []

    for index, dataset in enumerate(all_datasets):
        subsets = pre.split_by_gradient(
            dataframe=dataset,
            column="V_in",
            periods=1,
            sign=-1,
            min_length=50_000,
            reset_index=True,
        )
        for subset in subsets:
            measurement_cycles.append(subset)

else:

    test_measurement_cycles = pre.split_by_gradient(
        dataframe=test,
        column="V_in",
        periods=1,
        sign=-1,
        min_length=50_000,
        reset_index=True,
    )

In [ ]:
if SHOW:
    if not TESTING:

        # visualize one random dataset for data validation
        for dataset in measurement_cycles:
            print(dataset.attrs["path"])
            vis.plot_columns_as_subplots(dataset,
                                        dataset.columns)

        
    else:
        for dataset in test_measurement_cycles:
            # visualize the program testing set
            print(dataset.attrs["path"])
            vis.plot_columns_as_subplots(dataset, dataset.columns)

In den Messungen der Drehzahlen zeigen sich Ausreißer. Über eine lineare Regression mittels der `LinearRegression` aus dem Package Scikit-learn (Pedregosa et al., 2011) werden deren Positionen im Datensatz ermittelt und korrigiert. Als Korrekturwert dient der, von der `LinearRegression` ermittelte Soll-Wert. Die Modul-Funktion *clean_outliers()* erlaubt auch die Entfernung der Ausreißer.

In [ ]:
if not TESTING:

    for index, dataset in enumerate(measurement_cycles):
        # calculate fft frequencies and magnitudes alongside the median rpms
        clean_data = pre.clean_outliers(
            dataframe=dataset,
            column_name="Measured_RPM",
            window_size=None,
            std_multiplier=2,
            discard=True,
        )
        measurement_cycles[index] = clean_data

else:

    for index, dataset in enumerate(test_measurement_cycles):
        clean_data = pre.clean_outliers(
            dataframe=dataset,
            column_name="Measured_RPM",
            window_size=None,
            std_multiplier=3,
            discard=True,
        )
        test_measurement_cycles[index] = clean_data

In [ ]:
if SHOW:
    if not TESTING:

        # visualize one random dataset for data validation
        for dataset in measurement_cycles:
            print(dataset.attrs["path"])
            vis.plot_columns_as_subplots(dataset,
                                        dataset.columns)

        
    else:
        for dataset in test_measurement_cycles:
            # visualize the program testing set
            print(dataset.attrs["path"])
            vis.plot_columns_as_subplots(dataset, dataset.columns)

##### **Datentransformation**
Das FCNN von Mey et al. verarbeitet die Sensordaten von Sensor Nr. 1. Diese werden vorab in den Frequenzraum überführt. Hierbei werden jeweils die Datenmenge einer Sekunde &mdash; $n=4096$ Messwerte &mdash; Fourier-transformiert und die jeweils 2048 physikalisch relevanten Fourier-Koeffizienten weiterverwendet (Mey et al., 2020, Abschnitt B). <br>
NumPy (Harris et al., 2020) bietet zur Berechnung der positiven Frequenzanteile die Funktion `numpy.rfft()`, die in diesem Projekt implementiert ist. Sie löst die Diskrete Fourier-Transformation (DFT, Gleichung 1) für reelles Eingabesignal mit den Messwerten $a_m$. 

$$A_k = \frac{1}{n} \cdot \sum^{n-1}_{m=0} a_m \cdot e^{-2\,\pi\,i\,\frac{m\,k}{n}} \qquad \text{mit} \quad k = 0,~\dots,~n-1 \qquad \qquad (1)$$

Pro Sekunde des ursprünglichen Datensatzes gibt die `numpy.rfft()` 2049 reellwertige Fourier-Koeffizienten $A_k$. Das erste Element bildet den sog. "Zero Frequency Term", die Summe des gesamten Signals. Dieser Term wird verworfen. Die übrigen 2048 Werte bilden die Eingabe für das NN. <br>
Das gesamte Eingangssignal wird Fourier-transformiert, wobei sich die Fenster nicht überlagern. Aufgrund der Halbierung der Anzahl von Messwerten speichert ein neues Dataset die transformierten Daten. Für eine spätere Zuordnung wird mit derselben Fenstergröße der Median der Drehzahlen berechnet. Der neue Datensatz erhält eine Kopie der `DataFrame.attrs` des ursprünglichen Datensatzes. Den ursprünglichen Dateinamen ergänzt ein Suffix "_fft". Der Eintrag "sample_rate" wird auf 2048 Hz gesetzt.

In [ ]:
if not TESTING:

    for index, dataset in enumerate(measurement_cycles):
        # calculate fft frequencies and magnitudes alongside the median rpms
        fft_dataset = pre.calculate_dft(
            dataset,
            column_name=["Vibration_1", "Vibration_2", "Vibration_3"],
            normalize=False,
        )
        rpms = pre.median(dataset, column="Measured_RPM", stretch=True)
        fft_dataset["rpm"] = rpms[::2]
        measurement_cycles[index] = fft_dataset

else:
    
    for index, dataset in enumerate(test_measurement_cycles):
        fft_data = pre.calculate_dft(dataset, column_name="Vibration_*", normalize=False)
        rpms = pre.median(dataset, column="Measured_RPM", stretch=True)
        fft_data["rpm"] = rpms[::2]
        test_measurement_cycles[index] = fft_data

In [ ]:
if SHOW:
    if not TESTING:

        # describe one random dataset for data validation
        display(measurement_cycles[CONTROL_INDEX].describe())
        vis.plot_fft_spectrogram(data_frame=measurement_cycles[CONTROL_INDEX],
                                    figsize=(14, 6))

    else:
        
        # program testing set
        for dataset in test_measurement_cycles:
            display(dataset.describe())
            vis.plot_fft_spectrogram(data_frame=dataset,
                                    figsize=(14, 6))

##### **Datenskalierung**
Vor der Eingabe in das NN werden die Fourier-transformierten Daten skaliert. Mey et al. verwendeten hierfür "Robust Scaling" jeweils auf die Datenpunkte einer Sekunde. <br>
Das Programm nutzt hierzu den `RobustScaler` der Library Scikit-learn (Pedregosa et al., 2011). Der `RobustScaler` skaliert die Gesamtheit der $p=2048$ Fourier-Koeffizienten eines Fensters $\mathcal{A}$ mit einem Interquantilabstand (IRQ) zwischen dem 5%-Quantil $A_{0.05}$ und dem 95%-Quantil $A_{0.95}$ (vergl. Gl. 2).

$$A^*_k = \frac{A_k - \text{median}~{\mathcal{A}}}{A_{0.95} - A_{0.05}} \qquad \text{mit} \qquad \mathcal{A}=\{A_0,~\dots~A_k,~\dots~A_{p-1}\} \qquad \qquad (2)$$

In [ ]:
if not TESTING:

    for index, dataset in enumerate(measurement_cycles):
        measurement_cycles[index] = pre.scale_robust(dataset, column_name="*magnitude*")

else:
    for index, dataset in enumerate(test_measurement_cycles):
        scaled_data = pre.scale_robust(dataset, column_name="*magnitude*")
        test_measurement_cycles[index] = scaled_data

In [ ]:
if not TESTING:

    # describe one random dataset for data validation
    display(measurement_cycles[CONTROL_INDEX].describe())
    vis.plot_fft_spectrogram(data_frame=measurement_cycles[CONTROL_INDEX],
                                 figsize=(14, 6))

else:
    
    # program testing set
    for dataset in test_measurement_cycles:
        display(dataset.describe())
        vis.plot_fft_spectrogram(data_frame=dataset,
                                 figsize=(14, 6))

# WIP
Werte, die außerhalb der IQR liegen, liegen in den robust skalierten Daten außerhalb des Wertebereichs [-1, 1]. Die Funktion `pre.clean_quantiles` setzt diese Werte auf den Wert des Quantils. 

In [ ]:
if not TESTING:

    for index, dataset in enumerate(measurement_cycles):
        measurement_cycles[index] = pre.clean_quantiles(
            dataset,
            column_name="*magnitude*",
            quantiles=(0.05, 0.95),
            window_size=2048,
            discard=False,
        )

else:
    for index, dataset in enumerate(test_measurement_cycles):
        test_measurement_cycles[index] = pre.clean_quantiles(
            dataset,
            column_name="*magnitude*",
            quantiles=(0.05, 0.95),
            window_size=2048,
            discard=False,
        )

Abschließend speichert die Funktion `save_dataset()` des Moduls `data_loading` die Datensätze in CSV-Dateien. Hierzu legt die Funktion im Ordner *data/* den Unterordner *processed/`UUID`* an. In diesem Ordner erstellt sie eine YAML-Datei, in der die `DataFrame.attrs` jedes im gleichen Ordner gespeicherten Datensatzes eingetragen werden.

In [ ]:
# saving the preprocessed data
if not TESTING:

    for dataset in measurement_cycles:
        dl.save_dataset(dataset, uuid=UUID)

else:
    for dataset in test_measurement_cycles:
        dl.save_dataset(dataset, uuid=UUID)


#### **Quellen**

##### **Literatur**
Harris, C. R., Millman, K. J., Van der Walt, S. J., Gommers, R., Virtanen, P., Cournapeau, D., Wieser, E., Taylor, J., Berg, S., Smith, N. J., Kern, R., Picus, M., Hoyer, S., Van Kerkwijk, M. H., Brett, M., Haldane, A., Del Río, J. F., Wiebe, M., Peterson, P., . . . Oliphant, T. E. (2020). Array programming with NumPy. Nature, 585(7825), 357–362. https://doi.org/10.1038/s41586-020-2649-2

Mey, O., Neudeck, W., Schneider, A. & Enge-Rosenblatt, O. (2020). Machine learning-based unbalance detection of a rotating shaft using vibration data. In 2020 25th IEEE International Conference on Emerging Technologies and Factory Automation (ETFA) (S. 1610–1617). IEEE. https://doi.org/10.1109/etfa46521.2020.9212000

Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M. & Duchesnay, É. (2011). Scikit-learn: Machine Learning in Python. Journal Of Machine Learning Research, 12, 2825–2830. https://doi.org/10.5555/1953048.2078195

##### **Online**
Kuhlen, N., Beißner, L., Bergermann, T. & Wallrad, L. (2024). AI Gear Fault Detection (Version V1) [Computer Software]. https://github.com/ninakuhlen/ai-gear-fault-detection

Mey, O., Neudeck, W., Schneider, A. & Enge-Rosenblatt, O. (2022, 23. Februar). Vibration Analysis on Rotating Shaft. Kaggle. Abgerufen am 20. Oktober 2024, von https://www.kaggle.com/datasets/jishnukoliyadan/vibration-analysis-on-rotating-shaft

NumPy Developers. (o. D.). Discrete Fourier Transform (Numpy.FFT) — NUMPY v2.2 Manual. https://numpy.org/doc/stable/reference/routines.fft.html#module-numpy.fft

Scikit-learn Developers. (o. D.). RobustScaler. Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
